In [1]:
# import logging
import sys

# stdout=sys.stdout
sys.path.append('../../')
import os
import random
import time

import matplotlib
import matplotlib.pyplot as plt
import mplfinance as mpf
# pip install --upgrade mplfinance
# sys.stdout=stdout
import numpy as np
# import numpy as np
import pandas as pd
from JohnsonUtil import commonTips as cct
from JohnsonUtil import johnson_cons as ct
from JSONData import sina_data
from JSONData import tdx_data_Day as tdd
from mplfinance.original_flavor import candlestick_ohlc

# from JSONData import tdx_hdf5_api as h5a

%matplotlib widget
plt.rc('font', family='SimHei', size=13)
matplotlib.get_backend()
# jupyter nbextension enable --py widgetsnbextension

os.environ['NUMEXPR_MAX_THREADS'] = '12'
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
import nest_asyncio

nest_asyncio.apply()
from sklearn.preprocessing import StandardScaler


[DEBUG] Path Mode: Python Script (__file__). Path: D:\MacTools\WorkFile\WorkSpace\pyQuant3\stock_standalone\JohnsonUtil
使用本地配置: D:\MacTools\WorkFile\WorkSpace\pyQuant3\stock_standalone\JohnsonUtil\global.ini
log_f: instock_tk.log


In [2]:

# code = '920274'
# # Example usage
# # price_data = [100, 95, 97, 99, 102, 105, 108, 111,1]
# resample = 'd'
# df = tdd.get_tdx_Exp_day_to_df(
#     code, dl=ct.Resample_LABELS_Days[resample], resample=resample)
# # .sort_index(ascending=True).reset_index()

# df_feat = df.query('upper > 0')[['open', 'close','upper','EVAL_STATE','trade_signal']]


# 特征标准化/归一化

# 特征筛选或降维

In [3]:
def generate_df_vect_daily_features_safe(df, lastdays=5):
    """多股票，生成每只股票最近 lastdays 的特征字典列表"""
    features_list = []
    cols_map = {
        'open': 'lasto',
        'high': 'lasth',
        'low': 'lastl',
        'close': 'lastp',
        'vol': 'lastv',
        'upper': 'upper',
        'ma5d': 'ma5',
        'ma20d': 'ma20',
        'ma60d': 'ma60',
        'perlastp': 'perc',
        'perd': 'per'
    }
    
    for code, df_stock in df.groupby(level=0):
        feat = {'code': str(code)}
        df_stock = df_stock.sort_index(level=1)  # 按日期升序
        
        for da in range(1, lastdays+1):
            for col, prefix in cols_map.items():
                if col in df_stock.columns and len(df_stock) >= da:
                    feat[f'{prefix}{da}d'] = df_stock[col].iloc[-da]
                else:
                    feat[f'{prefix}{da}d'] = 0
            
            # eval / signal
            for suffix in ['eval', 'signal']:
                colname = f'{suffix}{da}d'
                if colname in df_stock.columns and len(df_stock) >= da:
                    feat[colname] = df_stock[colname].iloc[-da]
                else:
                    feat[colname] = 0
        features_list.append(feat)
    
    return pd.DataFrame(features_list)

In [4]:
code_l=['920274','300342','300696', '603091', '605167']

resample='d'

# df = tdd.get_tdx_exp_all_LastDF_DL(code_l, dt=ct.Resample_LABELS_Days[resample], resample=resample)

df = tdd.get_append_lastp_to_df()
# df_feat = tdd.generate_df_vect_daily_features(df,lastdays=cct.compute_lastdays)
# 排除非数值列（code 和 date）
# 假设 df_feat 中有 trade_signal


initdx :0 b1>:5460 it:0.4 tdx_last_df:2.99 TDD: 5464 resample:d

  0%|                                                                                         | 0/5464 [00:00<?, ?it/s]

  0%|                                                                                       | 0.00/5.46k [00:0…

KeyboardInterrupt: 

In [ ]:
df[:2]

In [ ]:
target_cols = ['trade_signal', 'EVAL_STATE']  # 如果有多个目标列
df_target = df[target_cols]

In [ ]:
df_target

In [ ]:
df

In [ ]:
# -*- coding: utf-8 -*-
import pandas as pd
from sklearn.preprocessing import StandardScaler

# ============================================
# 1. 设定参数
# ============================================
lastdays = 5  # 提取最近 N 天特征

# ============================================
# 2. 获取最近交易日数据
# 假设 df 的 index 是 MultiIndex: (code, date)
# ============================================
# latest_date = df.index.get_level_values(1).max()
# print("最新交易日:", latest_date)

# # 只保留最新交易日及历史 lastdays-1 日
df_recent = df.groupby(level=0).apply(
    lambda x: x.sort_index(level=1).tail(lastdays)
).reset_index(level=0, drop=False)  # 保留 code

# ============================================
# 3. 生成矢量化特征
# ============================================

# df_feat = generate_df_vect_daily_features_safe(df_recent, lastdays=lastdays)

# # 过滤掉 upper <= 0 的行
# df_feat = df_feat.query('upper1d > 0')

# # ============================================
# # 4. 特征标准化
# # ============================================
# scaler = StandardScaler()

# # signal 和 eval 不做标准化
# exclude_cols = ['code'] + [f'{s}{d}d' for d in range(1,lastdays+1) for s in ['eval','signal']]
# feat_cols = [c for c in df_feat.columns if c not in exclude_cols]

# df_feat_scaled = df_feat.copy()
# df_feat_scaled[feat_cols] = scaler.fit_transform(df_feat[feat_cols])

# # ============================================
# # 5. 输出结果
# # ============================================
# print(df_feat_scaled.head())

# # df_feat_scaled 可以直接作为模型输入


# 假设 df_feat 是 generate_df_vect_daily_features 返回的 DataFrame
df_feat = pd.DataFrame(generate_df_vect_daily_features_safe(df, lastdays=cct.compute_lastdays))
df_feat = df_feat.query('upper1d > 0')

In [ ]:


# 标准化特征列（不包括 code 和目标列）
feat_cols = [c for c in df_feat.columns if c not in ['code'] + target_cols]
df_feat_scaled = df_feat.copy()
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_feat_scaled[feat_cols] = scaler.fit_transform(df_feat[feat_cols])

# 将目标列重新合并
# 重置索引，让 code 变成列
df_target_reset = df_target.reset_index()
df_feat_scaled_reset = df_feat_scaled.reset_index(drop=True)  # 原来的索引丢掉

# 将目标列合并
df_feat_scaled_reset[target_cols] = df_target_reset[target_cols]


In [ ]:
target_cols,df_target,df_feat_scaled_reset

In [ ]:
df_feat_scaled_reset[target_cols]

## 特征与标签准备（Supervised Learning）

In [ ]:
# 假设 trade_signal 是目标
X = df_feat_scaled_reset.drop(columns=['code','trade_signal','EVAL_STATE'], errors='ignore')
y = df_feat_scaled_reset['trade_signal']  # 或者 'EVAL_STATE'

### 然后可以直接拆训练集/测试集：

In [ ]:
from sklearn.model_selection import train_test_split

# 拆分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 查看训练集和测试集的大小
print("X_train:", X_train.shape)
print("X_test :", X_test.shape)
print("y_train:", y_train.shape)
print("y_test :", y_test.shape)

# 查看前几行数据
print(X_train.head())
print(y_train.head())
